In [40]:
import pandas as pd

import numpy as np

In [8]:
fileName = "toronto_merged.xlsx"

toronto_neighs = pd.read_excel(fileName)

In [9]:
toronto_neighs.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Explore neighborhoods data and visualize them on the map

In [10]:
## count how many boroughs and neighborhood groups the dataset has

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_neighs['Borough'].unique()),
        toronto_neighs.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [45]:
# import required libraries

from geopy.geocoders import Nominatim

import folium # map rendering library

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


In [12]:
## use geopy to get latitude and longtitude of Toronto

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
# create map of New York using latitude and longitude values with neighborhoods superimposed on top

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighs['Latitude'], toronto_neighs['Longitude'], toronto_neighs['Borough'], toronto_neighs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Link to Foursquare and get info about venues in different neighborhoods

In [16]:
## define foursquare credentials

CLIENT_ID = 'AECVBA3XRV42QNIDW4HRTA1YPWEINXTOE5FT5VD3E215X2UH' # your Foursquare ID
CLIENT_SECRET = '30ESAB4SZDGTFKE00BYBN5U0LS1L5MGX2HHMHHJGRFDLUGQM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AECVBA3XRV42QNIDW4HRTA1YPWEINXTOE5FT5VD3E215X2UH
CLIENT_SECRET:30ESAB4SZDGTFKE00BYBN5U0LS1L5MGX2HHMHHJGRFDLUGQM


In [24]:
LIMIT = 100 # limit of number of venues returned by Foursquare API


radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d4c9eac342adf00387d9cfb'},
 'response': {'headerLocation': 'Birch Cliff',
  'headerFullLocation': 'Birch Cliff, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.6971570045, 'lng': -79.25863612686784},
   'sw': {'lat': 43.688156995499995, 'lng': -79.27106007313215}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '56d51743498e60da346470e2',
       'name': 'The Birchcliff',
       'location': {'address': '1666 Kingston Road',
        'crossStreet': 'Birchcliff Avenue',
        'lat': 43.69166644406541,
        'lng': -79.26453158481682,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.69166644406541,
          'lng': -79.26453158481682}],
        'di

In [18]:
## create a funcion to get nearby venues for all neighborhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names=toronto_neighs['Neighborhood'],
                                   latitudes=toronto_neighs['Latitude'],
                                   longitudes=toronto_neighs['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [27]:
print(toronto_venues.shape)
toronto_venues.head()

(2250, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum


In [29]:
## check counts of venues at each neighborhood group

toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",10,10,10,10,10,10


### Analyze neighborhood venues data

In [30]:
## analyze the neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
brooklyn_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
toronto_onehot.shape

(2250, 279)

In [32]:
## Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#brooklyn_grouped.sort_values(by = ["Yoga Studio"], ascending = False)

In [33]:
## print each neighborhood with the 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2             Clothing Store  0.25
3               Skating Rink  0.25
4  Middle Eastern Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                             venue  freq
0                       Playground  0.33
1                      Coffee Shop  0.33
2                             Park  0.33
3                Accessories Store  0.00
4  Molecular Gastronomy Restaurant  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store   0.2
1           Pizza Place   0.2
2              Pharmacy   0.1
3  Fast Food Restaurant   0.1
4   

                        venue  freq
0                        Park  0.50
1                 Pizza Place  0.25
2           Convenience Store  0.25
3  Modern European Restaurant  0.00
4                 Music Store  0.00


----Emery, Humberlea----
                 venue  freq
0       Baseball Field   1.0
1    Accessories Store   0.0
2  Monument / Landmark   0.0
3          Music Venue   0.0
4          Music Store   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.12
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.08
3   Japanese Restaurant  0.03
4              Tea Room  0.03


----First Canadian Place, Underground city----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.07
2   Restaurant  0.04
3   Steakhouse  0.04
4        Hotel  0.04


----Flemingdon Park, Don Mills South----
              venue  freq
0  Asian Restaurant  0.10
1       Coffee Shop  0.10
2        Beer Store  0.10
3               Gym  0.10
4    Discount St

                             venue  freq
0                       Playground   0.5
1                Convenience Store   0.5
2  Molecular Gastronomy Restaurant   0.0
3                      Music Venue   0.0
4                      Music Store   0.0


----Silver Hills, York Mills----
                 venue  freq
0            Cafeteria   1.0
1    Accessories Store   0.0
2  Monument / Landmark   0.0
3          Music Venue   0.0
4          Music Store   0.0


----St. James Town----
                venue  freq
0         Coffee Shop  0.06
1  Italian Restaurant  0.05
2                Café  0.05
3          Restaurant  0.05
4               Hotel  0.05


----Stn A PO Boxes 25 The Esplanade----
                  venue  freq
0           Coffee Shop  0.11
1                  Café  0.04
2            Restaurant  0.04
3                 Hotel  0.03
4  Fast Food Restaurant  0.03


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2   Italian Resta

In [38]:
## put this into pandas data frame

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,American Restaurant,Hotel,Burger Joint,Restaurant,Gym
1,Agincourt,Clothing Store,Breakfast Spot,Lounge,Skating Rink,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Coffee Shop,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Fried Chicken Joint,Pharmacy,Coffee Shop,Sandwich Place,Beer Store,Fast Food Restaurant,Golf Course,Gluten-free Restaurant
4,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Coffee Shop,Pool,Pub,Sandwich Place,Skating Rink,Pharmacy,Gym,Dive Bar


### Cluster Neighborhoods

In [37]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([8, 8, 2, 3, 3, 3, 1, 3, 8, 1])

In [53]:
kmeans.labels_

array([8, 8, 2, 3, 3, 3, 1, 3, 8, 1, 3, 1, 8, 2, 8, 1, 2, 3, 8, 3, 1, 1,
       8, 8, 3, 8, 0, 1, 3, 8, 3, 3, 1, 1, 8, 1, 8, 8, 8, 1, 8, 2, 6, 8,
       1, 8, 2, 1, 8, 1, 8, 1, 1, 0, 8, 3, 8, 8, 2, 2, 8, 3, 8, 2, 8, 1,
       1, 2, 9, 8, 7, 8, 3, 8, 2, 2, 8, 8, 3, 8, 8, 5, 8, 8, 1, 3, 1, 3,
       8, 8, 2, 3, 3, 3, 4, 3, 3, 3, 3, 8, 2])

In [43]:
## Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_neighs

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,8.0,Fast Food Restaurant,Print Shop,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.0,Golf Course,History Museum,Bar,Yoga Studio,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,8.0,Pizza Place,Rental Car Location,Medical Center,Mexican Restaurant,Electronics Store,Intersection,Breakfast Spot,Cosmetics Shop,Event Space,Comfort Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Insurance Office,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,8.0,Caribbean Restaurant,Fried Chicken Joint,Bank,Thai Restaurant,Athletics & Sports,Bakery,Hakka Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


### Test One Cluster

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0.0,Golf Course,History Museum,Bar,Yoga Studio,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
94,Etobicoke,0.0,Bank,Golf Course,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
